# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv("output_data/cities.csv")
cities_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
locations = cities_df[["Lat", "Lng"]]
weight = cities_df['Humidity'].astype(int)

# Plot Heatmap
fig = gmaps.figure()

# Create Heatmap layer
Heatmap_layers = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

# Add Heatmap layer to map
fig.add_layer(Heatmap_layers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Drop any rows that don't contain all three conditions. Conditions to be met: a. Maximum temperature lower than 80 degrees but higher than 70. b. Wind speed less than 10 mph. c.Zero cloudiness.

new_cities = cities_df.dropna()

optimum_weather = new_cities.loc[(new_cities['Cloudiness'] > 0) & (new_cities['Wind Speed'] < 10)] 
optimum_temp_weather = optimum_weather.loc[(optimum_weather['Max Temp'] > 70) & (optimum_weather['Max Temp'] <= 80)]
optimum_temp_weather

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
12,12,avarua,100,CK,1558378756,94,-21.21,-159.78,71.60,2.24
25,25,kantang,20,TH,1558378759,94,7.42,99.52,78.80,6.93
44,44,puerto ayora,26,EC,1558378762,86,-0.74,-90.35,79.00,5.99
59,59,bontang,54,ID,1558378765,99,0.12,117.47,74.22,2.62
87,87,ambilobe,2,MG,1558378770,92,-13.19,49.05,74.76,4.52
91,91,bella vista,62,AR,1558378771,80,-28.51,-59.05,75.66,7.29
100,100,sorong,95,ID,1558378773,93,-0.86,131.25,77.28,2.01
104,104,mbigou,70,GA,1558378773,94,-1.90,11.91,72.60,1.63
106,106,hilo,40,US,1558378755,69,19.71,-155.08,79.00,4.70
153,153,porto novo,20,BR,1558378783,53,-23.68,-45.44,77.00,4.70


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df =  pd.DataFrame(optimum_temp_weather, columns=["City", "Country", "Lat", "Lng"])
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
12,avarua,CK,-21.21,-159.78,
25,kantang,TH,7.42,99.52,
44,puerto ayora,EC,-0.74,-90.35,
59,bontang,ID,0.12,117.47,
87,ambilobe,MG,-13.19,49.05,
91,bella vista,AR,-28.51,-59.05,
100,sorong,ID,-0.86,131.25,
104,mbigou,GA,-1.90,11.91,
106,hilo,US,19.71,-155.08,
153,porto novo,BR,-23.68,-45.44,


In [7]:
#creates a list to capture hotel_names to insert into dataframe
Hotel_Name = []

# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# iteration using for loop
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    try:
        Hotel_Name.append(name_address['results'][0]['name'])
    except IndexError:
        Hotel_Name.append(np.nan)
        
hotel_df['Hotel Name']= Hotel_Name
hotel_df.dropna()

,City,Country,Lat,Lng,Hotel Name
12,avarua,CK,-21.21,-159.78,The Edgewater Resort and Spa
25,kantang,TH,7.42,99.52,โรงแรมกันตังปาร์ค(Kantang Park Hotel)
44,puerto ayora,EC,-0.74,-90.35,Hotel Solymar Galapagos
59,bontang,ID,0.12,117.47,Hotel Garuda Bontang
87,ambilobe,MG,-13.19,49.05,Hotel National
91,bella vista,AR,-28.51,-59.05,Hotel Río Arriba
100,sorong,ID,-0.86,131.25,Swiss-Belhotel Sorong
106,hilo,US,19.71,-155.08,Grand Naniloa Hotel Hilo - a DoubleTree by Hilton
153,porto novo,BR,-23.68,-45.44,Hotel Pousada Paradise
159,manado,ID,1.49,124.85,Four Points by Sheraton Manado


In [8]:
hotel_df.to_csv('Hotels.csv')

In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)

# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…